Solve

\begin{align}
\min_{x,y,z} \quad & 0.5(\sum_{i=1}^4 (x_i - d_{1i})^2 + (y_i - d_{2i})^2 + (z_i - d_{3i})^2 + (x_i - y_i)^2 + (y_i - z_i)^2) \\
\text{s.t.} \quad & x_1 = x_2 \\
& x_2 = x_3 \\
& x_3 = x_4 \\
& y_1 = y_2 \\
& y_3 = y_4
\end{align}

In [1]:
import numpy as np
import cvxpy as cp

In [2]:
d = np.array([[1, 2, 3, 4], [-1, 1, 3, 10], [10, 15, 20, 25]])
n = 4

In [28]:
xv = cp.Variable(n)
yv = cp.Variable(n)
zv = cp.Variable(n)

objective = cp.Minimize(0.5*(cp.sum_squares(xv - d[0]) + cp.sum_squares(yv - d[1]) + cp.sum_squares(zv - d[2]) + cp.sum_squares(xv-yv) + cp.sum_squares(yv-zv)))

cons = [xv[0] == xv[1], xv[1] == xv[2], xv[2] == xv[3], yv[0] == yv[1], yv[2] == yv[3]]

prob = cp.Problem(objective, cons)

result = prob.solve()

print(xv.value)
print(yv.value)
print(zv.value)

[3.40625 3.40625 3.40625 3.40625]
[4.6625 4.6625 7.4625 7.4625]
[ 7.33125  9.83125 13.73125 16.23125]


In [29]:
# Print dual variables
for con in cons:
    print(con.dual_value)

-2.15
-4.3
-2.65
-3.25
-5.250000000000001


In [30]:
d

array([[ 0,  0,  1,  2],
       [ 0,  4,  0,  8],
       [10, 15, 20, 25]])

In [6]:
A = np.array([[3, -1, 0], [-1, 4, -1], [0, -1, 3]])
Ainv = np.linalg.inv(A)
print(Ainv)

[[0.36666667 0.1        0.03333333]
 [0.1        0.3        0.1       ]
 [0.03333333 0.1        0.36666667]]


In [31]:
class fusedQuad():

    def __init__(self, data):
        self.data = data['d']
        self.shape = data['shape']
        self.x = self.data.copy()

    def prox(self, x, tau=1.0):
        self.x[:-1] = x
        self.x = Ainv @ (self.data + tau * self.x)
        return self.x[:-1]


In [22]:
d[:,0]

array([ 0,  0, 10])

In [19]:
ff = fusedQuad({'d': d[:,0], 'shape': 2})

In [20]:
y = np.zeros(2)
for i in range(10):
    y = ff.prox(y)  
    print(y)

[0.66666667 2.        ]
[1.02222222 2.4       ]
[1.16740741 2.48      ]
[1.2211358 2.496    ]
[1.24011193 2.4992    ]
[1.24665064 2.49984   ]
[1.24887288 2.499968  ]
[1.24962216 2.4999936 ]
[1.24987363 2.49999872]
[1.24995779 2.49999974]


In [21]:
ff.x

array([1.24995779, 2.49999974, 6.25004247])

In [40]:
resolvents = [fusedQuad({'d': d[:,i], 'shape': 2}) for i in range(4)]
itrs = 1000
num_stages = 2
stage_dims = [1, 1]
x = np.zeros((num_stages, 4))
v = np.zeros((num_stages, 4))
vLx = np.zeros(num_stages)
gamma = 0.5
stages = [0, 1]
L = [{(1, 0): 1, (2, 1): 1, (3,0):1, (3, 2): 1}, {(1,0): 2, (3,2): 2}]
W = [{(0, 1): -1, (0, 3): -1, (1, 0): -1, (1, 2): -1, (2, 1): -1, (2, 3): -1, (3, 0): -1, (3, 2): -1}, {(0, 1): -2, (1, 0): -2, (2, 3): -2, (3, 2): -2}]
links = [[[], [0], [1], [0, 2]], [[], [0], [], [2]]]
Wlinks = [[[1, 3], [0, 2], [1, 3], [0, 2]], [[1], [0], [3], [2]]]
for itr in range(itrs):
    for i in range(4):
        for s in stages:
            vLx[s] = v[s,i] + sum(L[s][i,j] * x[s,j] for j in links[s][i])
        x[:,i] = resolvents[i].prox(vLx)
        # print('Itr', itr, 'x', i, x[:,i])
    for i in range(4):
        delta = 0.0
        for s in stages:
            delta_s = sum(W[s][i,j]*x[s,j] for j in Wlinks[s][i]) + 2.0*x[s,i]
            delta += delta_s**2
            v[s, i] = v[s, i] - gamma*delta_s
    if delta**0.5 < 1e-6:
        break

for i in range(4):
    print('itrs', itr, 'x', x[:,i], 'v', i, v[:,i])
        # print('Itr', itr, 'v', i, v[:,i])

itrs 41 x [3.40624978 4.66250036] v 0 [5.55624928 7.91250101]
itrs 41 x [3.40625049 4.66249993] v 1 [ 2.15000123 -7.91250101]
itrs 41 x [3.40625015 7.46249975] v 2 [-1.64999986 12.71249931]
itrs 41 x [3.40624968 7.46250005] v 3 [ -6.05625066 -12.71249931]


In [37]:
print(xv.value)
print(yv.value)


[3.40625 3.40625 3.40625 3.40625]
[4.6625 4.6625 7.4625 7.4625]
